In [1]:
!nvidia-smi #this command acts as a task manager between nvidia gpu and the user. Shows GPU related details (system management information)




Tue Oct  7 04:39:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
!git clone https://github.com/Tanishk-Singh/Transformers.git

Cloning into 'Transformers'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 4), reused 6 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 5.95 KiB | 1.98 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [5]:
%cd Transformers

/content/drive/MyDrive/Transformers


In [6]:
!git config user.name "Tanishk-Singh"
!git config user.email "tanwartanishk5@gmail.com"

In [7]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [8]:
!pip install autoawq transformers accelerate

# autoawq - main quantisation library
# transformers - hugging face library to download latest llm models
# accelerate - hugging face library to load large models efficently on cpu/gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autoawq: filename=autoawq-0.2.9-py3-none-any.whl size=115106 sha256=a0f98f1a8b596a9f88cb9ed94e083dce8e6c6d5e2dce0a572f7f794e009f7be8
  Stored in directory: /root/.cache/pip/wheels/45/1a/7b/7314b3a958454e8ce349f600829a3f0a6a05aeebf987be1e16
Successfully built autoawq


In [9]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

/usr/local/lib/python3.12/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes

In [10]:
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print("AutoAWQ imported successfully!")

PyTorch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
AutoAWQ imported successfully!


In [13]:
!mkdir -p models/awq_quantized

In [21]:
# Clear the corrupted cache
!rm -rf /root/.cache/huggingface/hub/models--facebook--opt-1.3b

# Force re-download with explicit parameters
from transformers import AutoConfig

model_path = "facebook/opt-1.3b"

print("Downloading OPT-1.3B model (this will take 2-3 minutes)...")

# Download with force_download to ensure clean download
model = AutoAWQForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    trust_remote_code=True,
    force_download=False,  # Will use cache if valid, re-download if corrupt
    resume_download=True   # Resume if interrupted
)

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True
)

print(f"✅ Model loaded successfully!")
print(f"📊 Memory: {model.get_memory_footprint() / 1e9:.2f} GB")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

LICENSE.md: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

OSError: Error no file named model.safetensors found in directory /root/.cache/huggingface/hub/models--facebook--opt-1.3b/snapshots/3f5c25d0bc631cb57ac65913f76e22c2dfb61d62.

In [23]:
# Nuclear option - completely remove cache and force fresh download
!rm -rf /root/.cache/huggingface/hub/models--facebook--opt-1.3b

# Also clear any partial downloads
!rm -rf /root/.cache/huggingface/hub/.locks

# Now download with force_download=True
from huggingface_hub import snapshot_download
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "facebook/opt-1.3b"

print("Forcing fresh download of OPT-1.3B...")

# Download model files first
snapshot_download(
    repo_id=model_path,
    cache_dir="/root/.cache/huggingface/hub",
    force_download=True,
    resume_download=False
)

# Now load the model
model = AutoAWQForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    use_safetensors=False # Added this line
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

print(f"✅ Model loaded!")
print(f"📊 Memory: {model.get_memory_footprint() / 1e9:.2f} GB")

Forcing fresh download of OPT-1.3B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

LICENSE.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tf_model.h5:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

TypeError: transformers.models.auto.auto_factory._BaseAutoModelClass.from_pretrained() got multiple values for keyword argument 'use_safetensors'

In [36]:
!ls -la

total 259
-rw------- 1 root root 239874 Oct  7 05:40 AWQ_Implementation.ipynb
drwx------ 8 root root   4096 Oct  7 05:39 .git
-rw------- 1 root root   1096 Oct  7 04:51 LICENSE
drwx------ 3 root root   4096 Oct  7 05:16 models
-rw------- 1 root root   4253 Oct  7 04:51 quantization_min_max_percentile_range.ipynb
-rw------- 1 root root     82 Oct  7 04:51 README.md
-rw------- 1 root root   9731 Oct  7 04:51 understanding_quantization.ipynb


In [37]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	AWQ_Implementation.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [52]:
!git add .

In [53]:
!git push

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 23.44 KiB | 2.60 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        locations: